In [1]:
import getpass
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
model = ChatOpenAI(model="gpt-3.5-turbo")


In [2]:
# we will persist the messages in memory
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langchain_core.messages import BaseMessage, AIMessage
from langgraph.graph.message import add_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import Sequence
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

# lets add a system prompt to help the LLM work better
prompt = ChatPromptTemplate.from_messages(
    [(
        "system",
        "you talk like a DSP advertising operations expert working in Germany. Answer all questions to the best of your ability in {language}."
    ),
     MessagesPlaceholder(variable_name="messages")
    ]
)

def call_openai_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_openai_model)

# add the workflow to the memory saver
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [3]:
# we will add a config to pass to the runnable to enable multiple conversatin threads
config = {
    "configurable": {"thread_id": "abcd12345"}
}

In [4]:
#we will invoke the application here
query = "Hi I am Dozie"
language = "English"
input_messages = [HumanMessage(content=query)]
for chunk, metadata in app.stream({"messages": input_messages, "language": language}, config, stream_mode="messages"):
    if isinstance(chunk, AIMessage):
        print(chunk.content, end="|")


|Hello| Do|zie|,| how| can| I| assist| you| today|?||

In [5]:
# seeing if the model can remember the name
query1 = "What is my name?"
language = "Deutsch"
input_messages = [HumanMessage(content=query1)]
for chunk, metadata in app.stream({"messages": input_messages, "language": language}, config, stream_mode="messages"):
    if isinstance(chunk, AIMessage):
        print(chunk.content, end="|")

|De|in| Name| ist| Do|zie|.||

In [6]:
# by changing the thread_id, we can see that the model does not remember the name
config = {
    "configurable": {"thread_id": "abcd12345678"}
}
input_messages = [HumanMessage(content=query1)]
for chunk, metadata in app.stream({"messages": input_messages, "language": language}, config, stream_mode="messages"):
    if isinstance(chunk, AIMessage):
        print(chunk.content, end="|")

|Ent|sch|uld|igung|,| ich| habe| keine| Informationen| über| de|inen| Namen|.| Wie| kann| ich| sonst| weiter|h|elf|en|?||